In [1]:
import pandas as pd
import os
import xml.etree.ElementTree as ET
import warnings
import re

warnings.filterwarnings("ignore", category=FutureWarning)

def aplicar_mascara_contato(contato):
    if pd.notna(contato):
        contato_numerico = re.sub(r'[^\d.]', '', str(contato))
        
        if re.match(r'\d+(\.\d+)?$', contato_numerico):
            contato_numerico = re.sub(r'(\d{2})(\d{4,})(\d{4})', r'\1 \2-\3', contato_numerico)
            contato_string = str(contato_numerico)
            return contato_string[:-2]
    return ''

def criar_dataframe_sumarizado(df):
    regional = df['REGIONAL'].tolist()
    razao_social = df['DISTRIBUIDOR'].tolist()
    responsavel = df['RESPONSAVEL'].tolist()
    municipio_uf = df['MUNICIPIO / UF'].tolist()
    populacao = df['POPULACAO'].tolist()
    contato_responsavel = df['CONTATO1'].tolist()
    contato_distribuidor = df['CONTATO2'].tolist()
    endereco = df['ENDERECO'].tolist()

    categorias = []
    for reg in regional:
        categoria = reg.split(' ', 1)[0] if isinstance(reg, str) else ''
        categorias.append(categoria)

    max_length = max(len(regional), len(razao_social), len(responsavel), len(municipio_uf), len(populacao), len(contato_responsavel), len(contato_distribuidor), len(endereco), len(categorias))

    regional.extend([None] * (max_length - len(regional)))
    razao_social.extend([None] * (max_length - len(razao_social)))
    responsavel.extend([None] * (max_length - len(responsavel)))
    municipio_uf.extend([None] * (max_length - len(municipio_uf)))
    populacao.extend([None] * (max_length - len(populacao)))
    contato_responsavel.extend([None] * (max_length - len(contato_responsavel)))
    contato_distribuidor.extend([None] * (max_length - len(contato_distribuidor)))
    endereco.extend([None] * (max_length - len(endereco)))
    categorias.extend([None] * (max_length - len(categorias)))

    summary_df = pd.DataFrame({
        'REGIONAL': regional,
        'DISTRIBUIDOR': razao_social,
        'RESPONSAVEL': responsavel,
        'MUNICIPIO / UF': municipio_uf,
        'POPULACAO': populacao,
        'CONTATO1': contato_responsavel,
        'CONTATO2': contato_distribuidor,
        'ENDERECO': endereco,
        'CATEGORIA': categorias
    })

    # Replace NaN in 'RESPONSAVEL' column with 'EM BRANCO'
    #summary_df.dropna(subset=['REGIONAL'], inplace=True)

    summary_df['REGIONAL'].fillna('EM_BRANCO', inplace=True)
    summary_df['DISTRIBUIDOR'].fillna('EM_BRANCO', inplace=True)
    summary_df['RESPONSAVEL'].fillna('EM_BRANCO', inplace=True)
    summary_df['CONTATO1'].fillna('EM_BRANCO', inplace=True)
    summary_df['CONTATO2'].fillna('EM_BRANCO', inplace=True)
    summary_df['ENDERECO'].fillna('EM_BRANCO', inplace=True)
    summary_df['CATEGORIA'].fillna('EM_BRANCO', inplace=True)

    summary_df['CONTATO1'] = summary_df['CONTATO1']#.apply(aplicar_mascara_contato)
    summary_df['CONTATO2'] = summary_df['CONTATO2']#.apply(aplicar_mascara_contato)

    unique_razao_social = summary_df['DISTRIBUIDOR'].unique()
    color_mapping = {razao: f"{hash(razao) % 0xffffff:06x}ff" for razao in unique_razao_social}
    summary_df['COR'] = summary_df['DISTRIBUIDOR'].map(color_mapping)

    return summary_df

def ler_arquivo_kml(file_path):
    """Lê arquivos KML em um diretório e retorna os dados relevantes."""
    df_data = []
    tree = ET.parse(file_path)
    root = tree.getroot()

    for placemark in root.findall('.//{http://www.opengis.net/kml/2.2}Placemark'):
        name = placemark.find('{http://www.opengis.net/kml/2.2}name').text
        coordenadas = placemark.find('.//{http://www.opengis.net/kml/2.2}coordinates').text
        descricao = placemark.find('.//{http://www.opengis.net/kml/2.2}description').text
        coordenadas = ' '.join(coordenadas.split())

        df_data.append({'MUNICIPIO': name, 'DESCRICAO': descricao, 'COORDENADAS': coordenadas })
    return df_data

def processar_arquivos_kml(diretorio):
    """Processa todos os arquivos KML no diretório e retorna um DataFrame."""
    df_data = []
    
    for estado in os.listdir(diretorio):
        estado_path = os.path.join(diretorio, estado)

        for tipo in os.listdir(estado_path):
            tipo_path = os.path.join(estado_path, tipo)
            
            if 'kml' in tipo and os.path.isdir(tipo_path):
                for file_name in os.listdir(tipo_path):
                    if file_name.endswith('.kml'):
                        file_path = os.path.join(tipo_path, file_name)
                        dados = ler_arquivo_kml(file_path)
                        df_data.extend(dados)
    return df_data

def merge_arquivos(data_frame, diretorio):
    df_distribuidores = criar_dataframe_sumarizado(data_frame)
    df_municipios_data = processar_arquivos_kml(diretorio)
    
    df_municipios = pd.DataFrame(df_municipios_data)

    df_distribuidores.dropna(subset=['MUNICIPIO / UF'], inplace=True)

    df_merged = pd.merge(df_distribuidores, df_municipios, how='left', left_on='MUNICIPIO / UF', right_on='DESCRICAO')

    return df_merged

def gerar_estilos(categoria, cor, transparencia, espessura):
    if categoria == 'EM_BRANCO':
        cor = 'F1EE8Eff'
    else:
        cor = cor

    return f'''
    <Style id="poly-{cor}-1200-77-nodesc-normal">
      <LineStyle>
        <color>FFFFFFFF</color>
        <width>{espessura}</width>
      </LineStyle>
      <PolyStyle>
        <color>{cor}</color>
        <fill>1</fill>
        <outline>1</outline>
      </PolyStyle>
    </Style>
    <Style id="poly-{cor}-1200-77-nodesc-highlight">
      <LineStyle>
        <color>FFFFFFFF</color>
        <width>{transparencia}</width>
      </LineStyle>
      <PolyStyle>
        <color>{cor}</color>
        <fill>1</fill>
        <outline>1</outline>
      </PolyStyle>
    </Style>
    <StyleMap id="poly-{cor}-1200-77-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#poly-{cor}-1200-77-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#poly-FFFFFFFF-1200-77-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
'''

def gerar_placemark(contato_responsavel, populacao, distribuidor, responsavel, endereco, municipio_uf, cor, coordenadas, contato_distribuidor):
    return f'''
    <Placemark>
      <name>
          {municipio_uf}\nPOPULAÇÃO: {populacao}
      </name>
      <description>
          DISTRIBUIDOR: {distribuidor}\nCONTATO DISTRIBUIDOR: {contato_distribuidor}\nRESPONSÁVEL: {responsavel.upper()}\nCONTATO RESPONSÁVEL: {contato_responsavel}\nENDEREÇO: {endereco} 
      </description>
      <styleUrl>#poly-{cor}-1200-77-nodesc</styleUrl>
      <Polygon>
        <outerBoundaryIs>
          <LinearRing>
            <tessellate>1</tessellate>
            <coordinates>
              {coordenadas}
            </coordinates>
          </LinearRing>
        </outerBoundaryIs>
      </Polygon>
    </Placemark>
'''

def dados_template(dados_template, transparencia, espessura):
    template = f'''<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
  <Document>
    <name>Distribuidores</name>
'''

    cores_set = set()
    for categoria, categoria_data in dados_template.groupby('CATEGORIA'):
        template += f'<Folder><name>{categoria}</name>\n'
        
        for regional, regional_data in categoria_data.groupby('REGIONAL'):
            template += f'<Folder><name>{regional}</name>\n'

            for distribuidor, distribuidor_data in regional_data.groupby('DISTRIBUIDOR'):
                template += f'<Folder><name>{distribuidor}</name>\n'

                for municipio_data in distribuidor_data.itertuples():
                    cor = getattr(municipio_data, 'COR', None)
                    if cor and cor not in cores_set:
                        distribuidor = getattr(municipio_data, 'DISTRIBUIDOR', None)
                        responsavel = getattr(municipio_data, 'RESPONSAVEL', None)
                        endereco = getattr(municipio_data, 'ENDERECO', None)
                        contato_distribuidor = getattr(municipio_data, 'CONTATO2', None)
                        contato_responsavel = getattr(municipio_data, 'CONTATO1', None)
                        populacao = getattr(municipio_data, 'POPULACAO', None)
                        template += gerar_estilos(categoria, cor, transparencia, espessura)
                        cores_set.add(cor)

                    populacao = getattr(municipio_data, 'POPULACAO', None)
                    endereco = getattr(municipio_data, 'ENDERECO', None)
                    contato_distribuidor = getattr(municipio_data, 'CONTATO2', None)
                    contato_responsavel = getattr(municipio_data, 'CONTATO1', None)

                    distribuidor = getattr(municipio_data, 'DISTRIBUIDOR', None)
                    responsavel = getattr(municipio_data, 'RESPONSAVEL', None)
                    municipio_uf = getattr(municipio_data, 'DESCRICAO', None)
                    coordenadas = getattr(municipio_data, 'COORDENADAS', None)

                    cor = getattr(municipio_data, 'COR', None)

                    template += gerar_placemark(contato_responsavel, populacao, distribuidor, responsavel, endereco, municipio_uf, cor, coordenadas, contato_distribuidor)

                template += '</Folder>\n'

            template += '</Folder>\n'

        template += '</Folder>\n'

    template += '''
  </Document>
</kml>
'''
    return template

def create_combined_template(df_merged, output_file_path, transparencia, espessura):
    template = dados_template(df_merged, transparencia, espessura)
    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.write(template)

def limpa_diretorio(diretorio):
    try:
        if os.path.exists(diretorio):
            arquivos = os.listdir(diretorio)

            for arquivo in arquivos:
                caminho_completo = os.path.join(diretorio, arquivo)
                if os.path.isfile(caminho_completo):
                    os.remove(caminho_completo)
                elif os.path.isdir(caminho_completo):
                    os.rmdir(caminho_completo)
    except Exception as e:
        print(f"Ocorreu um erro durante a limpeza do diretório: {e}")

def main():
    
    transparencia = 0.8
    espessura = 0.2

    caminho_arquivo = r'C:\Users\bruno\OneDrive\Área de Trabalho\Fluxo de trabalho\mapeamento\excel\BRUNO.xlsx'    
    output_file_path = r'C:\Users\bruno\OneDrive\Área de Trabalho\Fluxo de trabalho\mapeamento\saida\distribuidores_kml\unificado\distribuidores.kml'
    diretorio_a_limpar = r'C:\Users\bruno\OneDrive\Área de Trabalho\Fluxo de trabalho\mapeamento\saida\distribuidores_kml\unificado'
    diretorio_kml = r'C:\Users\bruno\OneDrive\Área de Trabalho\Fluxo de trabalho\mapeamento\kml-brasil-master\lib\2010\municipios'

    limpa_diretorio(diretorio_a_limpar)

    df = pd.read_excel(caminho_arquivo, sheet_name='BASE LIMPA')

    df_result = merge_arquivos(df, diretorio_kml)    
    create_combined_template(df_result, output_file_path, transparencia, espessura)

if __name__ == "__main__":
    main()

    